# Thymus ageing atlas | T/NK cell compartment: Differential gene expression analysis between early paed and adult samples

In [ ]:
import os
import sys
import session_info
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad
import hdf5plugin

# Add repo path to sys path (allows to access scripts and metadata from repo)
repo_path,_ = os.path.split(os.path.split(os.getcwd())[0])
repo_path = '/lustre/scratch126/cellgen/team205/lm25/thymus_projects/thymus_ageing_atlas/T_NK_compartment'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/lustre/scratch126/cellgen/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts')

# Autoreload custom scripts
%load_ext autoreload
%autoreload 2

# Define paths
plots_path = f'{repo_path}/plots/'
data_path = f'{repo_path}/data/'
model_path = os.path.join(repo_path, 'models')
general_data_path = '/lustre/scratch126/cellgen/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/data'

print('Dir for plots: {}'.format(plots_path))
print('Dir for data: {}'.format(data_path))

# Formatting
from matplotlib import font_manager
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
plt.style.use('/lustre/scratch126/cellgen/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
from plotting.utils import plot_grouped_boxplot, calc_figsize

## Load data

In [ ]:
# Load data
object_version = 'v8_2024-11-07'
adata = ad.read_h5ad(f'{data_path}/objects/rna/thyAgeing_tSplit_scvi_{object_version}.zarr')

# For multiple text columns, you can use a loop
for column in adata.obs.columns:
    if pd.api.types.is_object_dtype(adata.obs[column]):
        try:
            adata.obs[column] = adata.obs[column].str.decode('utf-8')
        except AttributeError:  # This catches columns that are not bytes type
            pass

# Add knn predictions to adata (original HTSA reference does not have uncertainties)
ct_anno = pd.read_csv(f'{data_path}/objects/rna/thyAgeing_tSplit_scvi_{object_version}_curatedAnno_v7.csv', index_col = 0, dtype=str)
adata.obs = adata.obs.join(ct_anno)
adata = adata[~pd.isna(adata.obs['taa_l5']),:]

# Update metadata
latest_meta_path = get_latest_version(dir = f'{general_data_path}/metadata', file_prefix='Thymus_ageing_metadata')
latest_meta = pd.read_excel(latest_meta_path)
update_obs(adata, latest_meta, on = 'index', ignore_warning = True)

adata

In [ ]:
adata.obs.groupby('age_group').agg(n_cells = ('age_group', 'count'),
                                    n_donors = ('donor', 'nunique'),
                                    n_samples = ('sample', 'nunique'))  

In [ ]:
adata.obs_names.duplicated().sum()

In [ ]:
# Define columns
col_age_group = 'age_group'
col_cell_type = 'taa_l4'
col_cell_type_fine_levels = get_ct_levels(col_cell_type, taa_l1 = ['NK', 'T'])

## Differential gene expression analysis (DESeq2)

### Pseudobulking

In [ ]:
# Add nhood enrichment
nhood_anno = pd.read_csv(f'{data_path}/objects/rna/thyAgeing_tSplitxTissue_scvi_v1_2024-11-29_nhoodEnrichment.csv', index_col = 0)
nhood_anno.index = nhood_anno.index.str.replace('-\d', '', regex=True)
adata.obs = adata.obs.join(nhood_anno)

In [ ]:
from utils import aggClusters, add_batch_pca

factors = [col_age_group, 'sex','nhood_enrichment']
contrast = ['nhood_enrichment', 'tissue', 'blood']
agg_key_cols = [col_cell_type, 'donor', 'nhood_enrichment']
meta_cols = ['age', 'age_months', 'study']

min_cells = 10
min_rep = 3

# Add metadata columns to adata
for c in agg_key_cols:
    adata = adata[~pd.isna(adata.obs[c]),:]
print('Size of adata for aggregating clusters: {}'.format(adata.shape))
adata.obs['agg_key'] = adata.obs[agg_key_cols].astype(str).agg("__".join, axis=1)

# Aggregate across clusters and complete metadata
meta_cols = list(set(meta_cols + agg_key_cols + factors))
agg_adata = aggClusters(adata, lognorm=None,cluster_key='agg_key',raw = 'X', preserve_meta=meta_cols)

# Remove pseudobulks with too few cells (n < 10)
agg_adata = agg_adata[agg_adata.obs.n_cells >= min_cells,:]
agg_adata = agg_adata[~pd.isna(agg_adata.obs[col_age_group]),:]

In [ ]:
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

from adjustText import adjust_text
from matplotlib.backends.backend_pdf import PdfPages

reps_per_group = agg_adata.obs.groupby([col_cell_type, contrast[0]]).size().to_frame('n_reps').reset_index().pivot(index=col_cell_type, columns=contrast[0], values='n_reps').fillna(0)
excluded_ct = reps_per_group.loc[(reps_per_group[contrast[1]] < min_rep) | (reps_per_group[contrast[2]] < min_rep)].index

if len(excluded_ct) > 0:
    print(f'Excluding {len(excluded_ct)} cell types with less than {min_rep} replicates: {excluded_ct.tolist()}.')
    agg_adata = agg_adata[~agg_adata.obs[col_cell_type].isin(excluded_ct),:]

contrast_mod = [c.replace('-', '_') for c in contrast]
factors_mod = [c.replace('-', '_') for c in factors]
dea_res_dict = {}
for ct in agg_adata.obs[col_cell_type].unique():
    
    print(f'Running DE analysis for {ct}...')
    
    # Preparing metadata and counts
    metadata = agg_adata[agg_adata.obs[col_cell_type] == ct].obs[factors]
    counts = pd.DataFrame(agg_adata[agg_adata.obs[col_cell_type] == ct].X.toarray(), index=agg_adata[agg_adata.obs[col_cell_type] == ct].obs_names, columns=agg_adata[agg_adata.obs[col_cell_type] == ct].var_names)
    genes_to_keep = counts.columns[counts.sum(axis=0) >= 10]
    counts = counts[genes_to_keep]
    
    # Set up DeSeq dataset
    dds = DeseqDataSet(
        counts=counts,
        metadata=metadata,
        design = ' + '.join(factors_mod),
        min_replicates = min_rep,
        refit_cooks=True,
        n_cpus = 8,
    )
    
    # Estimate size factors and dispersion and fitting LFC
    dds.deseq2()
    
    # Run DE analysis for nhood effect
    dea_res = DeseqStats(dds, contrast=contrast_mod, inference=DefaultInference(n_cpus=8))
    dea_res.summary()
    dea_res_dict[ct] = dea_res.results_df

# Save as pickle
with open(f'{data_path}/analyses/phenoAnalysis/dea/thyAgeing_dea_taa_l4_tissueEffect.pkl', 'wb') as f:
    pkl.dump(dea_res_dict, f)

In [ ]:
deg_df = pd.concat(dea_res_dict).reset_index(names = ['cell_type', 'gene_name'])
deg_df.head()

In [ ]:
df = deg_df.copy()
df = df.loc[(df['padj'] < 0.05) & (abs(df['log2FoldChange']) > 1), :]
df['up_down'] = np.where(df['log2FoldChange'] > 0, 'up', 'down')
df = df.groupby(['cell_type', 'up_down']).size().unstack().fillna(0)
df.head()

In [ ]:
# Create volcano plots
with PdfPages(f'{plots_path}/phenoAnalysis/dea/thyAgeing_dea_tissueEffect_volcano.pdf') as pdf:
    for ct in df.index:
        sub_df = deg_df[deg_df['cell_type'] == ct].copy()
        
        # Create a column to indicate significant genes
        sub_df['is_signif'] = (sub_df['padj'] < 0.05) & (abs(sub_df['log2FoldChange']) >= 1.3)

        # Create the volcano plot
        plt.figure(figsize=calc_figsize(width = 60, height = 80))
        sns.scatterplot(data=sub_df, x='log2FoldChange', y=-np.log10(sub_df['pvalue']), hue='is_signif', legend=False)

        texts = []
        for i in range(sub_df.shape[0]):
            if sub_df.iloc[i]['is_signif']:
                texts.append(plt.text(sub_df.iloc[i]['log2FoldChange'], -np.log10(sub_df.iloc[i]['pvalue']), sub_df.iloc[i]['gene_name'], fontsize=8))
        adjust_text(texts, force_points=0.2, force_text=1)

        plt.axhline(y=-np.log10(0.05), color='grey', linestyle='--')
        plt.axvline(x=0, color='grey', linestyle='--')
        plt.xlabel('Log2 Fold Change')
        plt.ylabel('-Log10 p-value')
        plt.title(f'{ct}')
        
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()

Adult vs infant:

- IGF2BP3: lost in adults, required for thymic autonomy and self-renewal capacity of DN3 when deprived of progenitors (Martins, 2021), lined to leukemia and other cancers
- AREG: implicated in thymic regeneration post injury or induced involution (Nevo, 2024) secreted by Tregs (also in muscle injury repair -> (Lin, 2022)) and ILCs
- TXNIP: Studies have shown that TXNIP is associated with the development of allergen-specific and inflammatory memory Th2 cells (Liu, 2024), and its upregulation is critical for the formation of these memory cells (Kokubo et al., 2023). Furthermore, TXNIP has been found to promote human NK cell development (Persyn et al., 2022). In the context of T cell development, TXNIP restrains effector T cell and germinal center B cell expansion (Muri et al., 2020). Additionally, TXNIP induces growth arrest and enhances apoptosis in certain leukemia cells (Noura et al., 2020). TXNIP Maintains the Hematopoietic Cell Pool by Switching the Function of p53 under Oxidative Stress (Choi, 2013)
- CH25H (up) : Ch25h acts as a metabolic switch controlling T-cell mediated inflammation. Ch25h catalyzes cholesterol to generate 25-hydroxycholesterol (25OHC), which was subsequently released to the cellular milieu, functioning as a modulator of T cell response. Extracellular 25OHC suppressed cholesterol biosynthesis in T cells, inhibited cell growth, and induced nutrient deprivation cell death without releasing high-mobility group box 1 (HMGB1). This growth inhibitory effect was specific to actively proliferating cells with high cholesterol demand and was reversed when extracellular cholesterol was replenished. (Takahashi, 2021; Zhong, 2023)
- DDIT4 (up):

### Number of DEGs per transition

In [ ]:
contrasts = ['geriatric_vs_adult','adult_vs_paed', 'paed_vs_infant']
contrasts.reverse()
all_degs = {k:pd.read_csv(f'{data_path}/analyses/dea/thyAgeing_dea_{k}.csv') for k in contrasts}
all_degs = pd.concat(all_degs).reset_index(names=['contrast','remove']).drop(columns='remove')

all_degs = all_degs.loc[(all_degs['padj'] < 0.05) & (all_degs['log2FoldChange'].abs() >= 1.3)]
all_degs['cell_type'] = pd.Categorical(all_degs['cell_type'], categories=col_cell_type_fine_levels, ordered=True)
all_degs['up_down'] = ['up' if lfc > 0 else 'down' for lfc in all_degs['log2FoldChange']]

all_degs

In [ ]:
# Plot number of DEGs by cell type
df = all_degs.groupby(['contrast','cell_type'], observed=True).size().to_frame('n_genes').reset_index()
df['contrast'] = df['contrast'].str.replace('geriatric', 'aged')

with sns.plotting_context('paper', font_scale = 1.4):
    g = sns.catplot(data=df, x="cell_type", y="n_genes", hue="contrast", kind="bar", hue_order=['paed_vs_infant','adult_vs_paed', 'aged_vs_adult'],
                    height=5, aspect=2)
    g.set_xticklabels(rotation=90)
    g.set(yscale="log")
    
    g.set_xlabels('Cell type')
    g.set_ylabels('Number of DEGs')
    g._legend.set_title('Contrast')

    g.tight_layout()
    plt.savefig(f'{plots_path}/phenoAnalysis/dea/thyAgeing_nDegs_by_cell_type.png', dpi = 300, bbox_inches = 'tight')

## GSEA

In [ ]:
# Load DEA results
col_age_group = 'age_group'
contrast = [col_age_group, 'adult', 'paed']

df = pd.read_pickle(f'{data_path}/analyses/dea/thyAgeing_dea_{contrast[1]}_vs_{contrast[2]}.pkl')

df = pd.concat([v.results_df for k,v in df.items()], keys = df.keys(), names = ['cell_type']).reset_index().rename(columns = {'level_1':'gene_name'})
df.head()

GS schema: {TEST_GROUP}_vs_{REF_GROUP}_{DIR}

In [ ]:
import gseapy
import pickle 

# Retrieve human genesets from MsigDb
msig = gseapy.Msigdb()
gmt_c2_reactome = msig.get_gmt(category='c2.cp.reactome', dbver="2023.2.Hs")
gmt_c2_biocarta = msig.get_gmt(category='c2.cp.biocarta', dbver="2023.2.Hs")
gmt_c3_tft = msig.get_gmt(category='c3.tft', dbver="2023.2.Hs")
gmt_c7_immunesigdb = msig.get_gmt(category='c7.immunesigdb', dbver="2023.2.Hs")

# Specify run IDs and geneset dict
geneset_dict = {'c2_reactome': gmt_c2_reactome,
                'c2_biocarta': gmt_c2_biocarta,
                'c3_tft' : gmt_c3_tft,
                'c7_immunesigdb': gmt_c7_immunesigdb
                }

# Populate gsea dict by running GSEA on all DEA runs and genesets
# Rank genes and determine ties
all_res = {}
for ct in df['cell_type'].unique() :
    dea_res = df.loc[df['cell_type'] == ct,:]
    ranked_genes = dea_res[['gene_name', 'log2FoldChange']].set_index('gene_name')
    ties = 1- np.unique(ranked_genes['log2FoldChange']).shape[0]/ranked_genes.shape[0]

    # Run GSEA if ties <= 30%
    if (ranked_genes.shape[0] > 0) & (ties <= 0.3):

        all_res[ct] = {}
        for n,s in geneset_dict.items():
            
            pre_res = gseapy.prerank(rnk=ranked_genes, 
                        gene_sets=s,
                        threads=4,
                        min_size=5,
                        max_size=1000,
                        permutation_num=1000, # reduce number to speed up testing
                        outdir=None, # don't write to disk
                        seed=6,
                        verbose=True, # see what's going on behind the scenes
                        )
            
            pre_res = pre_res.res2d.loc[pre_res.res2d['FWER p-val'] < .05,:]
            
            if pre_res.shape[0] > 0 :
                all_res[ct][n] = pre_res
                
all_res = {k:pd.concat(v).reset_index().rename(columns = {'level_0':'geneset'}) for k,v in all_res.items()}
with open(f'{data_path}/analyses/dea/thyAgeing_gsea_{contrast[1]}_vs_{contrast[2]}.pkl', 'wb') as file:
    pickle.dump(all_res, file)
    
pd.concat(all_res).drop(columns = 'level_1').reset_index().rename(columns = {'level_0':'cell_type'}).drop(columns = 'level_1') \
    .to_csv(f'{data_path}/analyses/dea/thyAgeing_gsea_{contrast[1]}_vs_{contrast[2]}.csv', index = False)


In [ ]:
gsea_adult_infant = pd.read_csv(f'{data_path}/analyses/dea/thyAgeing_gsea_adult_vs_infant.csv')
gsea_adult_infant = gsea_adult_infant.loc[gsea_adult_infant['geneset'] != 'c7_immunesigdb']
gsea_adult_infant.columns

In [ ]:
freq_genesets = gsea_adult_infant.groupby('Term').size().to_frame(name = 'n').sort_values(by = 'n', ascending = False)
freq_genesets = freq_genesets.loc[freq_genesets['n'] >= 4,:]
freq_genesets = freq_genesets.index.tolist()
len(freq_genesets)

In [ ]:
gsea_adult_infant.groupby('Term').size() > 4
terms = (gsea_adult_infant.groupby('Term').size() > 4).loc[lambda x: x].index.tolist()
terms

In [ ]:
col_cell_type_fine_levels

In [ ]:
cell_types_dev_early = ['T_DN(early)',
 'T_DN(P)',
 'T_DN(Q)',
 'T_DN(late)',
 'T_DP(P)',
 'T_DP(Q)',
 'T_αβT(entry)',]
cell_types_dev_late = ['T_αβT(early)', 'T_Treg(agonist)',
 'T_CD8_naive',
 'T_CD4_naive',
 'T_Treg']
cell_types_recirc = ['T_CD8_naive_recirc',
 'T_CD8_rm',
 'T_CD8_em',
 'T_CD8_age-assoc',
 'T_CD4_act',
 'T_CD4_naive_recirc',
 'T_CD4_h',
 'T_CD4_fh',
 'T_CD4_r1',
 'T_CD4_em',
 'T_Treg_recirc',]

In [ ]:
import matplotlib.colors as mcolors
import textwrap
df = gsea_adult_infant.copy()
df = df.loc[df['cell_type'].isin(cell_types_recirc)]
df['geneset'] = df['geneset'].str.replace(')', ') ').str.replace('/', '/ ')
df['cell_type'] = pd.Categorical(df['cell_type'], categories=col_cell_type_fine_levels, ordered=True)
df['cell_type'].cat.remove_unused_categories(inplace=True)
df['abs_NES'] = df['NES'].abs()

# Filter according to frequency
#df = df.loc[df['Term'].isin((df.groupby('Term').size() >= 2).loc[lambda x: x].index.tolist()),:]

# Create a divergent colormap centered around 0
vmin, vmax, vcenter = df['NES'].min(), df['NES'].max(), 0
normalize = mcolors.TwoSlopeNorm(vcenter=vcenter, vmin=vmin, vmax=vmax)
cmap = sns.diverging_palette(220, 20, center="light", as_cmap=True, s=100)

xranges = df.groupby("geneset")["Term"].nunique()
xranges *= 1.1
g = sns.relplot(data=df, x='Term', y='cell_type', hue='NES', size='abs_NES',
                palette=cmap, hue_norm=normalize, height=4, aspect=4,
                col='geneset', col_order=xranges.index, legend=None,
                facet_kws={'sharey': True, 'sharex': False, 'gridspec_kws': dict(width_ratios=xranges, wspace=0.01)})

# Function to wrap text
def wrap_text(text, width=15):
    return '\n'.join(textwrap.wrap(text, width))

# Apply the text wrapping function to facet titles
g.set_titles("{col_name}", fontweight='bold')
for ax in g.axes.flat:
    title = ax.get_title()
    ax.set_title(wrap_text(title), fontweight='bold')

g.set_xticklabels(rotation=90)
g.set_xlabels('Gene')
g.set_ylabels('Cell type')
g.tight_layout()

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=normalize)
sm.set_array([])
g.figure.colorbar(sm, ax=g.axes, orientation='vertical', label='NES', pad=0.01)
plt.savefig(f'{plots_path}/phenoAnalysis/dea/thyAgeing_tCells_popularTerms_recircT_gsea_heatmap.png', dpi = 300, bbox_inches = 'tight')